<a href="https://datamics.com/de/courses/"><img src=../DATA/bg_datamics_top.png></a>

<em text-align:center>© Datamics</em>
# TensorFlow Klassifikation

## Daten

https://archive.ics.uci.edu/ml/datasets/pima+indians+diabetes

1. Titel: Pima Indianer Diabetes Datenbank

2. Quellen:
   (a) Ursprüngliche Eigentümer: Nationales Institut für Diabetes und Verdauungstrakt und
                        Nierenerkrankungen
   (b) Spender der Datenbank: Vincent Sigillito (vgs@aplcen.apl.jhu.edu)
                          Forschungszentrum, Leiter der RMI-Gruppe
                          Labor für Angewandte Physik
                          Die Johns Hopkins Universität
                          Johns Hopkins Straße
                          Lorbeer, MD 20707
                          (301) 953-6231
   (c) Eingangsdatum: 9. Mai 1990

3. Vergangene Nutzung:
    1. Smith, J.~W., Everhart, J.~E., Dickson, W.~C., Knowler, W.~C., \&
       Johannes,~R.~S. (1988). Verwendung des ADAP-Lernalgorithmus zur Prognose von
       das Auftreten von Diabetes mellitus.  In den Proceedings of the Symposium
       zu Computeranwendungen und medizinischer Versorgung}. (Seiten 261--265).  IEEE
       Computer Society Press.

       Die diagnostische, binär bewertete Variable, die untersucht wird, ist, ob die
       Patient zeigt Anzeichen von Diabetes nach Angaben der Weltgesundheitsorganisation
       Kriterien (d.h., wenn der 2-Stunden-Plasmaglukosespiegel mindestens 
       200 mg/dl bei jeder Besichtigungsuntersuchung oder bei Routineuntersuchungen
       Pflege).   Die Bevölkerung lebt in der Nähe von Phoenix, Arizona, USA.

       Ergebnisse: Ihr ADAP-Algorithmus macht eine reale Vorhersage zwischen
       0 und 1. dies wurde in eine binäre Entscheidung mit einem Cutoff von 
       0.448.  Mit 576 Trainingsinstanzen wird die Sensitivität und Spezifität der
       ihres Algorithmus lag bei 76% der verbleibenden 192 Instanzen.

4. Relevante Informationen:
      Bei der Auswahl dieser Instanzen wurden mehrere Einschränkungen gemacht, nämlich
      eine größere Datenbank.  Insbesondere sind alle Patienten hier weiblich an der
      mindestens 21 Jahre alt von Pima Indianer Erbe.  ADAP ist ein adaptives Lernen
      Routine, die digitale Analoge von perzeptronähnlichen
      Geräte.  Es ist ein einzigartiger Algorithmus; siehe das Papier für Details.

5. Anzahl der Instanzen: 768

6. Anzahl der Attribute: 8 plus Klasse 

    7. Für jedes Attribut: (alle numerisch bewertet)
       1. Anzahl der Schwangerschaften
       2. Plasmaglukosekonzentration a 2 Stunden im oralen Glukosetoleranztest
       3. Diastolischer Blutdruck (mm Hg)
       4. Dicke der Trizeps-Hautfalte (mm)
       5. 2-Stunden-Serum-Insulin (mu U/ml)
       6. Body Mass Index (Gewicht in kg/(Höhe in m)^2)
       7. Diabetes Stammbaumfunktion
       8. Alter (Jahre)
       9. Klassenvariable (0 oder 1)
       
8. Fehlende Attributwerte: Ja, 

9. Klassenverteilung: (Klassenwert 1 wird als "positiv getestet auf
   Diabetes")

   Klasse Wert Anzahl der Instanzen
   0 500
   1 268

10. Kurze statistische Analyse:

        Attributnummer:    Mittelwert:   Standardabweichung:
        1.                     3.8       3.4
        2.                   120.9       32.0
        3.                    69.1       19.4
        4.                    20.5       16.0
        5.                    79.8       115.2
        6.                    32.0       7.9
        7.                     0.5       0.3
        8.                    33.2       11.8




In [1]:
import pandas as pd

In [2]:
diabetes = pd.read_csv('pima-indians-diabetes.csv')

FileNotFoundError: File b'pima-indians-diabetes.csv' does not exist

In [ ]:
diabetes.head()

In [ ]:
diabetes.columns

### Daten bereinigen

In [ ]:
cols_to_norm = ['Number_pregnant', 'Glucose_concentration', 'Blood_pressure', 'Triceps',
       'Insulin', 'BMI', 'Pedigree']

In [ ]:
diabetes[cols_to_norm] = diabetes[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [ ]:
diabetes.head()

### Feature Spalten

In [ ]:
diabetes.columns 

In [ ]:
import tensorflow as tf

### Kontinuierliche Merkmale

* Anzahl der Schwangerschaften
* Plasmaglukosekonzentration a 2 Stunden im oralen Glukosetoleranztest
* Diastolischer Blutdruck (mm Hg)
* Triceps Hautfaltendicke (mm)
* 2-Stunden-Serum-Insulin (mu U/ml)
* Body Mass Index (Gewicht in kg/(Höhe in m)^2)
* Diabetes Stammbaumfunktion


In [ ]:
num_preg = tf.feature_column.numeric_column('Number_pregnant')
plasma_gluc = tf.feature_column.numeric_column('Glucose_concentration')
dias_press = tf.feature_column.numeric_column('Blood_pressure')
tricep = tf.feature_column.numeric_column('Triceps')
insulin = tf.feature_column.numeric_column('Insulin')
bmi = tf.feature_column.numeric_column('BMI')
diabetes_pedigree = tf.feature_column.numeric_column('Pedigree')
age = tf.feature_column.numeric_column('Age')

### Kategorische Features

Wenn du die Menge aller möglichen Merkmalswerte einer Spalte kennst und es nur wenige davon gibt, kannst du categorical_column_with_vocabulary_list verwenden. Wenn du die Menge der möglichen Werte im Voraus nicht kennst, kannst du  categorical_column_with_hash_bucket verwenden.

In [ ]:
assigned_group = tf.feature_column.categorical_column_with_vocabulary_list('Group',['A','B','C','D'])
# Alternative
# assigned_group = tf.feature_column.categorical_column_with_hash_bucket('Group', hash_bucket_size=10)

### Kontinuierlich in kategorisch umwandeln

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
diabetes['Age'].hist(bins=20)

In [ ]:
age_buckets = tf.feature_column.bucketized_column(age, boundaries=[20,30,40,50,60,70,80])

###  Zusammensetzen der Daten

In [ ]:
feat_cols = [num_preg ,plasma_gluc,dias_press ,tricep ,insulin,bmi,diabetes_pedigree ,assigned_group, age_buckets]

### Train Test Split

In [ ]:
diabetes.head()

In [ ]:
diabetes.info()

In [ ]:
x_data = diabetes.drop('Class',axis=1)

In [ ]:
labels = diabetes['Class']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_data,labels,test_size=0.33, random_state=101)

### Input Funktion

In [ ]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=1000,shuffle=True)

### Das Modell erstellen

In [ ]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols,n_classes=2)

In [ ]:
model.train(input_fn=input_func,steps=1000)

In [ ]:
# Useful link ofr your own data
# https://stackoverflow.com/questions/44664285/what-are-the-contraints-for-tensorflow-scope-names

## Evaluation

In [ ]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      y=y_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [ ]:
results = model.evaluate(eval_input_func)

In [ ]:
results

## Vorhersage

In [ ]:
pred_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [ ]:
# Voraussagen sind ein Generator!
predictions = model.predict(pred_input_func)

In [ ]:
list(predictions)

# DNN Klassifikator

In [ ]:
dnn_model = tf.estimator.DNNClassifier(hidden_units=[10,10,10],feature_columns=feat_cols,n_classes=2)

In [ ]:
# Oh ein Fehler. Sieh dir das Video an, um zu sehen, warum und wie man den Fehler beheben kann.
# https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/feature_column/feature_column.py
dnn_model.train(input_fn=input_func,steps=1000)

In [ ]:
embedded_group_column = tf.feature_column.embedding_column(assigned_group, dimension=4)

In [ ]:
feat_cols = [num_preg ,plasma_gluc,dias_press ,tricep ,insulin,bmi,diabetes_pedigree ,embedded_group_column, age_buckets]

In [ ]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=1000,shuffle=True)

In [ ]:
dnn_model = tf.estimator.DNNClassifier(hidden_units=[10,10,10],feature_columns=feat_cols,n_classes=2)

In [ ]:
dnn_model.train(input_fn=input_func,steps=1000)

In [ ]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      y=y_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [ ]:
dnn_model.evaluate(eval_input_func)

# Gut gemacht!